In [ ]:
import csv
import sys
import os
import pandas as pd
import sys
sys.path.append(r"e:\Coding Projects\Investment Analysis")

# There's a folder called csv_files in the same directory as this notebook, it has a file called Portfolio.csv
file_path = os.path.join(os.getcwd(), 'csv_files', 'Portfolio.csv')

def import_portfolio(file_path):
    try:
        with open(file_path, mode='r', newline='', encoding='utf-8-sig') as file:
            reader = csv.reader(file)
            rows = list(reader)

            # Find the header row (look for "Symbol" as the first column)
            header_row_idx = None
            for idx, row in enumerate(rows):
                if row and row[0].strip().lower() == "symbol":
                    header_row_idx = idx
                    break

            if header_row_idx is None:
                raise ValueError("Could not find header row with 'Symbol' in the CSV file.")

            headers = rows[header_row_idx]
            data_rows = rows[header_row_idx + 1:]

            # Define known summary/account labels (case-insensitive)
            summary_labels = [
                "futures positions market value",
                "futures cash",
                "cash & cash investments",
                "account total"
            ]

            # Separate positions and summary/account rows
            positions = []
            account_info = {}
            for row in data_rows:
                if len(row) != len(headers) or row[0].strip() == "":
                    continue
                label = row[0].strip()
                if label.lower() in summary_labels:
                    account_info[label] = dict(zip(headers, row))
                else:
                    positions.append(row)

            positions_df = pd.DataFrame(positions, columns=headers)
            return positions_df, account_info
    except FileNotFoundError:
        print(f"Error: The file {file_path} does not exist.")
        sys.exit(1)
    except Exception as e:
        print(f"An error occurred: {e}")
        sys.exit(1)


positions_df, account_info = import_portfolio(file_path)
futures_positions_market_value = account_info['Futures Positions Market Value']
futures_cash = account_info['Futures Cash']
cash_and_cash_investments = account_info['Cash & Cash Investments']
account_total = account_info['Account Total']

'''display(positions_df)
display(pd.DataFrame([futures_positions_market_value], index=[0]))
display(pd.DataFrame([futures_cash], index=[0]))
display(pd.DataFrame([cash_and_cash_investments], index=[0]))
display(pd.DataFrame([account_total], index=[0]))
'''

def parse_option_symbol(symbol):
    try:
        parts = symbol.split()
        if len(parts) >= 4:
            underlying = parts[0]
            expiration = parts[1]
            strike = parts[2]
            put_call = parts[3]
            return pd.Series([underlying, expiration, strike, put_call])
        else:
            # For non-option rows, fill with None
            return pd.Series([None, None, None, None])
    except Exception:
        return pd.Series([None, None, None, None])

positions_df[['Underlying Symbol', 'Expiration Date', 'Strike Price', 'Put / Call']] = positions_df['Symbol'].apply(parse_option_symbol)
#drop column 'Reinvest Capital Gains?' and 'Reinvest?'
positions_df.drop(columns=['Reinvest Capital Gains?', 'Reinvest?'], inplace=True, errors='ignore')

#display(positions_df)
#display(account_info)
#retreive rows whose valuse are 'XLV' in the Underlying Symbol column
xlv_positions = positions_df[positions_df['Underlying Symbol'] == 'XLV']
display(xlv_positions)
# Retreive rows whose values are 'XLV' in the Underlying Symbol column and 'Put' in the Put / Call column